In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 1248 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_type = "gemma" # llama, phi-3, gemma

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "gsarti/gemma-2-2b-rebus-solver-adapters", # MODEL OR ADAPTER FOLDER
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
stop_token_id = model.config.eos_token_id
if model_type == "gemma":
    stop_token = "<|eot_id|>"
    stop_token_id = tokenizer.encode(stop_token)[0]

In [ ]:
from tqdm import tqdm

if model_type == "llama":
    tokenizer.padding_side = "right"
elif model_type in ("phi-3", "gemma"):
    tokenizer.padding_side = "left"

In [ ]:
from datasets import load_dataset
dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["train.jsonl"], split = "train")

In [ ]:
# NOTA: qua dovrai sicuro cambiare un po' di roba

ex_idx = 5
example = dataset[ex_idx]["conversations"][0]
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": example["value"]}
    ],
    add_generation_prompt=True,
    return_tensors = "pt",
    padding=True,
    truncation=True,
)
inputs = inputs.to('cuda:0')

# l'idea secondo me è che `inputs` va passato a GRPOTrainer, ma controlla!

In [ ]:
# qua a occhio resta tutto tale e quale

from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # Increase to 4 for smoother training
    num_generations=6,  # Decrease if out of memory
    max_prompt_length=max_prompt_length,
    max_completion_length=500,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=...,
    args=training_args,
    train_dataset=...,
)

# QUA SECONDO ME DEVI RI-DEFINIRE UN TRAINING DATASET CON LA FORMATTAZIONE DI SOPRA

